<a href="https://colab.research.google.com/github/cmzwolf/JupyterVAMDCPortal/blob/main/portalTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VAMDC portal in a Jupyter Notebook 
This notebook implements in google-colab functionality similar to the ones of the VAMDC portal, in particular the part for requesting databases and dowloading data. **This is a first prototype**: the code will be factored into functions and new functionalities will be added

Let us start by a configuration phase, where we fetch all the required compoenents

In [1]:
# fetching the VAMDC Python libraries
!git clone https://github.com/notlaast/vamdclib.git 
%cd vamdclib

Cloning into 'vamdclib'...
remote: Enumerating objects: 181, done.
remote: Total 181 (delta 0), reused 0 (delta 0), pack-reused 181
Receiving objects: 100% (181/181), 166.20 KiB | 914.00 KiB/s, done.
Resolving deltas: 100% (85/85), done.
/content/vamdclib


In [2]:
# installing the VAMDC libraries as a python package (-e option is for interactive linking)
!pip install -e .

Obtaining file:///content/vamdclib
  Running setup.py develop for vamdclib


In [3]:
# installing dependencies for interacting with the VAMDC registries
!pip install suds-jurko

     |████████████████████████████████| 255 kB 7.1 MB/s 
  Created wheel for suds-jurko: filename=suds_jurko-0.6-py3-none-any.whl size=454320 sha256=44e84a1a22e7d5d29c66718c7c004c9535a8cf7a96860fc462b815c2e939c3bf
  Stored in directory: /root/.cache/pip/wheels/f4/75/2a/54353e9cd47f7dd7861856a750c5f99f05da1ccf624100b7a5
Successfully built suds-jurko


**We are ready to go!** Let us check the registries to see what nodes are registered

In [5]:
# quering the registries to get the node list
import pandas as pd
from vamdclib import nodes
from vamdclib import request
nl = nodes.Nodelist()

In [61]:
# display the nodes into a nice table, using Pandas data frame
%load_ext google.colab.data_table
nodeNames = []
nodesUrls = []
nodesIdentifiers = []
nodesMaintainers = []
for node in nl:
  nodeNames.append(node.name)
  nodesUrls.append(node.url)
  nodesIdentifiers.append(node.identifier)
  nodesMaintainers.append(node.maintainer)

df = pd.DataFrame(list(zip(nodeNames, nodesUrls, nodesIdentifiers, nodesMaintainers)),
               columns =['NodeName', 'NodeURL','nodeIdentifiers', 'nodeMaintainer'])
df

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


,NodeName,NodeURL,nodeIdentifiers,nodeMaintainer
0,Water internet Accessible Distributed Informat...,http://vamdc.saga.iao.ru/node/wadis/tap/,ivo://vamdc/wadis/vamdc-tap,faz@iao.ru
1,NIST Atomic Spectra Database,https://physics.nist.gov:8000/nodes/asd/tap/,ivo://vamdc/nist/vamdc-tap_12.07,yuri.ralchenko@nist.gov
2,CDMS,https://cdms.astro.uni-koeln.de/cdms/tap/,ivo://vamdc/cdms/vamdc-tap_12.07,endres@ph1.uni-koeln.de
3,MeCaSDa - Methane Calculated Spectroscopic Dat...,http://vamdc.icb.cnrs.fr/mecasda-12.07/tap/,ivo://vamdc/dijon-methane-lines,Vincent.Boudon@u-bourgogne.fr
4,GeCaSDa: Gemane Calculated Spectroscopic Database,http://vamdc.icb.cnrs.fr/gecasda/tap/,ivo://vamdc/dijon-GeH4-lines,cyril.richard@u-bourgogne.fr
5,Theoretical spectral database of polycyclic a...,http://vamdc-pah.oa-cagliari.inaf.it/tap/,ivo://vamdc/OA-Cagliari/PAH,gmulas@oa-cagliari.inaf.it
6,IDEADB - Innsbruck Dissociative Electron Attac...,https://ideadb.uibk.ac.at/tap/,ivo://vamdc/IDEADB,johannes.postler@uibk.ac.at
7,OACT - LASP Database,http://dblasp.oact.inaf.it/node1207/OACT/tap/,ivo://vamdc/OACatania/LASP,gle@oact.inaf.it
8,Carbon Dioxide Spectroscopic Databank 296K (VA...,http://lts.iao.ru/node/cdsd-296-xsams1/tap/,ivo://vamdc/cdsd-296,vip@lts.iao.ru
9,VALD (atoms),http://vald.astro.uu.se/atoms-12.07/tap/,ivo://vamdc/vald/uu/django,thomas.marquart@fysast.uu.se


We can now define the queries we would like submit. Each query will be submitted to the nodes we will select.

In [11]:
# define the set of queries to perform
queries =[]
# select HCO
queries.append("select species")
# select HCN
queries.append("select * where ((InchiKey = 'LELOWRISYMNNSU-UHFFFAOYSA-N'))")


In [13]:
# selecting the nodes to query by their indexe
selectedNodes = []
selectedNodesIndexes = [2,9]
for i in selectedNodesIndexes:
  currentNodeIdentifier = df['nodeIdentifiers'][i]
  currentNode = nl.getnode(currentNodeIdentifier)
  selectedNodes.append(currentNode)

In [14]:
# build all the request, considering the predefined set of queries and nodes
from vamdclib import request
requests = []

for currentNode in selectedNodes:
  for currentQuery in queries:
    req = request.Request()
    req.setnode(currentNode)
    req.setquery(currentQuery)
    requests.append(req)

In [15]:
# making head requests
from vamdclib import request
for req in requests:
  req.doheadrequest()

In [59]:
# Displaying the performed head queries 
headerColumns = ["VAMDC-COUNT-SPECIES", "VAMDC-COUNT-STATES", "VAMDC-TRUNCATED", "VAMDC-COUNT-MOLECULES", "VAMDC-COUNT-SOURCES", "VAMDC-APPROX-SIZE", "VAMDC-COUNT-RADIATIVE", "VAMDC-COUNT-ATOMS", "VAMDC-REQUEST-TOKEN", "Query"]
countSpecies = []
countStates =[]
truncated = []
countMolecules = []
countSources = []
approxSize = []
countRadiative = []
countAtoms = []
requestToken = []
submittedQueries = []
for req in requests:
  countSpecies.append(req.headers.get(headerColumns[0]))
  countStates.append(req.headers.get(headerColumns[1]))
  truncated.append(req.headers.get(headerColumns[2]))
  countMolecules.append(req.headers.get(headerColumns[3]))
  countSources.append(req.headers.get(headerColumns[4]))
  approxSize.append(req.headers.get(headerColumns[5]))
  countRadiative.append(req.headers.get(headerColumns[6]))
  countAtoms.append(req.headers.get(headerColumns[7]))
  requestToken.append(req.headers.get(headerColumns[8]))
  submittedQueries.append(req.query.Query)

HeadDF = pd.DataFrame(list(zip(countSpecies, countStates, truncated, countMolecules, countSources, approxSize, countRadiative, countAtoms, requestToken, submittedQueries)),
            columns = headerColumns   )


In [60]:
HeadDF

,VAMDC-COUNT-SPECIES,VAMDC-COUNT-STATES,VAMDC-TRUNCATED,VAMDC-COUNT-MOLECULES,VAMDC-COUNT-SOURCES,VAMDC-APPROX-SIZE,VAMDC-COUNT-RADIATIVE,VAMDC-COUNT-ATOMS,VAMDC-REQUEST-TOKEN,Query
0,1073,0,100,1062,0,3084.32,5494236,11,cdms:309d0fd7-7443-4c3c-a00f-a0fcfe9393d8:head,select species
1,6,766,100,6,16,1.23,936,0,cdms:af894d31-cff5-4fad-907b-15c79c38e7ed:head,select * where ((InchiKey = 'LELOWRISYMNNSU-UH...
2,309,None,None,None,None,None,None,309,vald:2070668d-5689-435f-804e-21db481d5d2d:head,select species
3,None,None,None,None,None,None,None,None,None,select * where ((InchiKey = 'LELOWRISYMNNSU-UH...


**The last line of the previous tab contains no data**: the reason is simple and depends on the header that each node return. Sometimes node use capital letters, sometimes lowercase letters. The previous code works with capital letters. It is up to the client code to adapt to the variety of behaviour of the nodes? Or should the nodes answer all the same way? **To discuss further in VAMDC...**

We are going to retrieve data only for the query indexed by 1. Once the data is fetched, we put them into a file. For naming this file, we will chose the Token of the query, since this is unique. 

In [69]:
# selecting the query indexed by 1 to retrieve data
requestToPerform = requests[1]

In [70]:
# running the query to get the results
result = requestToPerform.dorequest()

In [71]:
# we use the query token to name the file in a unique way (later we may rename it using the query store ID) and save the result XSAMS file
resultFileName = "/content/"+ HeadDF["VAMDC-REQUEST-TOKEN"][1]+".xsams"
with open(resultFileName, "wb") as text_file:
    text_file.write(result.Xml)

**Excellent!** Now open the file explorer on the left of this colab-notebook. Find your file. Click-left and donwload it. You have your XSAMS file! congrats. 

In [ ]:
!wget https://raw.githubusercontent.com/VAMDC/Processors/master/static/xsl/molecularxsams2html.xsl

doc	 molecularxsams2html.xsl  README    vamdclib
LICENSE  Output.xsams		  setup.py  vamdclib.egg-info


In [ ]:
!pwd

/content/vamdclib/vamdclib/vamdclib
